In [1]:
import pandas as pd
import csv
import os
import numpy as np
import cv2
pd.options.mode.chained_assignment = None

In [2]:
def read_data_with_subdirectorys(data_path, extension):
    videos_path_list = []
    print("List of all directories in '% s':" % data_path)

    for path, subdirs, files in os.walk(data_path):
        for name in files:
            if(extension is not None):
                if name.endswith(extension):
                    videos_path_list.append(os.path.join(path, name))
            else:
                videos_path_list.append(os.path.join(path, name))

    return videos_path_list

def is_au(string):
    txt = string
    x = txt.replace("+", "")
    return x.isdigit()


def read_concat_xml(path):
    
    df_results = pd.DataFrame(columns=['frame', 'start', 'end', 'video_name'])
    
    lista_xmls = read_data_with_subdirectorys(path, '.xml')
    file = None
    for file in lista_xmls:
        print(file)
        df = pd.read_xml(file)
        df = df.reset_index() #para pegar o frame
        df['index'] = df['index'] +1
        df = df.rename(columns={'index':'frame', 'v':'aus'})
        df['video_name'] = file.split('/')[-1].split('.')[0]
        print(df.shape)
        #print(df_results.shape)
        
        df_results = pd.concat([df_results, df], ignore_index=True)

    return df_results

In [3]:
path = "../data/outputs/Videos-Emely-Extra/predicts_squeezenet/"
df = read_concat_xml(path)
df = df.sort_values(by=['video_name', 'start'], ascending=(False, True))
df

List of all directories in '../data/outputs/Videos-Emely-Extra/predicts_squeezenet/':
../data/outputs/Videos-Emely-Extra/predicts_squeezenet/VID-20190313-WA0003.xml
(58, 5)
../data/outputs/Videos-Emely-Extra/predicts_squeezenet/VID-20190313-WA0005.xml
(31, 5)
../data/outputs/Videos-Emely-Extra/predicts_squeezenet/VID-20190313-WA0004.xml
(25, 5)


,frame,start,end,video_name,aus
58,1,0.000,0.266,VID-20190313-WA0005,0
59,2,0.233,5.042,VID-20190313-WA0005,Chega de briga nessa casa!
60,3,0.266,0.391,VID-20190313-WA0005,1+2
61,4,0.391,0.424,VID-20190313-WA0005,42
62,5,0.424,0.524,VID-20190313-WA0005,1+2+43
...,...,...,...,...,...
53,54,6.838,6.871,VID-20190313-WA0003,12+23+42
54,55,6.871,8.448,VID-20190313-WA0003,12+25
55,56,8.448,8.515,VID-20190313-WA0003,12+25
56,57,8.515,8.582,VID-20190313-WA0003,25


In [4]:
def define_frame_init_final(df3, fps_video):

    df3['frame_inicial'] = np.ceil((df3['start'])*fps_video).astype(int)
    #df3['frame_final'] = df3['frame_inicial'].shift(-1, fill_value=0)
    df3['frame_final'] = np.ceil((df3['end'])*fps_video).astype(int)
    df3['frame_final'] = df3['frame_final'].apply(lambda x: x-1 if x>0 else x) #diminundo 1 frame para evitar sobreposição

    #Eliminando a ocorrência com frase
    df3["au_valid"] = df3.apply(lambda row: is_au(row['aus']), axis=1) 
    df3 = df3.loc[df3.au_valid == True][['frame', 'start', 'end', 'video_name', 'aus', 'frame_inicial', 'frame_final']]
    
    return df3

def aus_distribution(df3):
    df_true = pd.DataFrame(columns = ['video_name', 'frame', 'aus'])
    frame_max = df3.frame_final.max()
    print(frame_max)

    for i in range(0, frame_max):
        
        row = df3.loc[(df3.frame_inicial <= i+1) & (df3.frame_final >= i+1)]
        new_row = {
            "video_name": row.video_name.item(),
            "frame": i+1,
            "aus": row.aus.item(),
        }

        df_true = df_true.append(new_row, ignore_index=True)
    return df_true
    


In [5]:
df_results = pd.DataFrame(columns = ['video_name', 'frame', 'aus'])

for i in df.video_name.unique():
        extensao = '.mp4'
        print(path+i+extensao)
        v_entry = cv2.VideoCapture(path+i+extensao, 0)
        #frames = int(v_entry.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = v_entry.get(cv2.CAP_PROP_FPS)
        df_frames = define_frame_init_final(df.loc[df.video_name == i], fps)
        df_aus = aus_distribution(df_frames)
        
        df_results = pd.concat([df_aus, df_results], ignore_index=True)

display(df_results)        
    

../data/outputs/Videos-Emely-Extra/predicts_squeezenet/VID-20190313-WA0005.mp4
151
../data/outputs/Videos-Emely-Extra/predicts_squeezenet/VID-20190313-WA0004.mp4
233
../data/outputs/Videos-Emely-Extra/predicts_squeezenet/VID-20190313-WA0003.mp4
282


,video_name,frame,aus
0,VID-20190313-WA0003,1,0
1,VID-20190313-WA0003,2,0
2,VID-20190313-WA0003,3,0
3,VID-20190313-WA0003,4,0
4,VID-20190313-WA0003,5,0
...,...,...,...
661,VID-20190313-WA0005,147,0
662,VID-20190313-WA0005,148,0
663,VID-20190313-WA0005,149,0
664,VID-20190313-WA0005,150,0


In [6]:
df_results.video_name.value_counts()

VID-20190313-WA0003    282
VID-20190313-WA0004    233
VID-20190313-WA0005    151
Name: video_name, dtype: int64

In [7]:
df_results.loc[df_results.video_name == 'VID-20190313-WA0003'].head(30)

,video_name,frame,aus
0,VID-20190313-WA0003,1,0
1,VID-20190313-WA0003,2,0
2,VID-20190313-WA0003,3,0
3,VID-20190313-WA0003,4,0
4,VID-20190313-WA0003,5,0
5,VID-20190313-WA0003,6,0
6,VID-20190313-WA0003,7,42
7,VID-20190313-WA0003,8,43
8,VID-20190313-WA0003,9,43
9,VID-20190313-WA0003,10,43


In [8]:
df_results.loc[df_results.video_name == 'VID-20190313-WA0004'].head(30)

,video_name,frame,aus
282,VID-20190313-WA0004,1,25+26
283,VID-20190313-WA0004,2,25+26
284,VID-20190313-WA0004,3,25+26
285,VID-20190313-WA0004,4,25+26
286,VID-20190313-WA0004,5,25+26
287,VID-20190313-WA0004,6,25+26
288,VID-20190313-WA0004,7,25+26
289,VID-20190313-WA0004,8,25+26
290,VID-20190313-WA0004,9,25+26
291,VID-20190313-WA0004,10,25+26


In [9]:
df_results.loc[df_results.video_name == 'VID-20190313-WA0005'].head(30)

,video_name,frame,aus
515,VID-20190313-WA0005,1,0
516,VID-20190313-WA0005,2,0
517,VID-20190313-WA0005,3,0
518,VID-20190313-WA0005,4,0
519,VID-20190313-WA0005,5,0
520,VID-20190313-WA0005,6,0
521,VID-20190313-WA0005,7,0
522,VID-20190313-WA0005,8,1+2
523,VID-20190313-WA0005,9,1+2
524,VID-20190313-WA0005,10,1+2


In [10]:
df_results.loc[df_results.video_name == 'VID-20190313-WA0005'].tail(30)

,video_name,frame,aus
636,VID-20190313-WA0005,122,1+2+28+51
637,VID-20190313-WA0005,123,1+2+28+51
638,VID-20190313-WA0005,124,1+2+28+51
639,VID-20190313-WA0005,125,1+2+28+51
640,VID-20190313-WA0005,126,1+2+28
641,VID-20190313-WA0005,127,1+2+28
642,VID-20190313-WA0005,128,1+2+24
643,VID-20190313-WA0005,129,1+2+22+25
644,VID-20190313-WA0005,130,1+2+22+25
645,VID-20190313-WA0005,131,1+2+22+25


In [11]:
df_results = df_results.sort_values(by=['video_name', 'frame'], ascending=(False, True))
df_results

,video_name,frame,aus
515,VID-20190313-WA0005,1,0
516,VID-20190313-WA0005,2,0
517,VID-20190313-WA0005,3,0
518,VID-20190313-WA0005,4,0
519,VID-20190313-WA0005,5,0
...,...,...,...
277,VID-20190313-WA0003,278,0
278,VID-20190313-WA0003,279,0
279,VID-20190313-WA0003,280,0
280,VID-20190313-WA0003,281,0


In [12]:
df_results = df_results.rename(columns={"aus": "ground truth"})
df_results.to_csv("../data/outputs/Videos-Emely-Extra/predicts_squeezenet/Videos-Emely-Extra-ground-truth.csv", sep=';')